# Package installation

In [2]:
!pip install openai-whisper
!sudo apt update && sudo apt install ffmpeg
!pip install pdfplumber openai pandas tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.5 MB/s

In [3]:
!pip install spacy
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 4.2 MB/s eta 0:00:00


In [4]:

!pip install openpyxl


In [5]:
!pip install transformers
!pip install tqdm

# import packages


In [6]:
import pandas as pd
import requests
from google.colab import drive
import os
import pdfplumber
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline, BertTokenizer, BertForSequenceClassification
import torch
import re

In [7]:
import spacy
from sentence_transformers import SentenceTransformer, util
from bertopic import BERTopic
import re
from spacy.pipeline import EntityRuler

In [8]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from tqdm import tqdm

# import data

In [12]:
drive.mount('/content/drive')
!ls '/content/drive/My Drive/BoE'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Codes				'Presentation Documents'
 Data				 Project_Plan
'Ideas and Research Documents'	'Russell Hunter Notebooks'
 Meetings			 Video_presentation
'Other Documents'


In [13]:
rec = '/content/drive/My Drive/BoE/Data/Unicredit/recordings/'

transcript_folder = '/content/drive/My Drive/BoE/Data/Unicredit/transcripts/'

output_folder = '/content/drive/My Drive/BoE/Data/Unicredit/extracted_outputs/'

os.makedirs(output_folder, exist_ok=True)

In [14]:
# q123 = os.path.join(rec, '1Q23-UniCredit-Conference-Call.mp3')
# q223 = os.path.join(rec, '2Q23-UniCredit-Conference-Call.mp3')
# q323 = os.path.join(rec, '3Q23-UniCredit-Conference-Call.mp3')
# q423 = os.path.join(rec, '4Q23-UniCredit-Conference-Call.mp3')

# q124 = os.path.join(rec, '1Q24-UniCredit-Conference-Call.mp3')
# q224 = os.path.join(rec, '2Q24-UniCredit-Conference-Call.mp4')
# q324 = os.path.join(rec, '3Q24-UniCredit-Conference-Call.mp3')
# q424 = os.path.join(rec, '4Q24-UniCredit-Conference-Call.mp4')


# extract info

bert

In [15]:
# model_name = "yiyanghkust/finbert-tone"
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name)
# finbert_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


# def add_finbert_sentiment(df, text_column="text"):
#     sentiments = []
#     scores = []
#     for text in df[text_column].fillna("").tolist():
#         try:
#             result = finbert_pipeline(text[:512])[0]  # Truncate if needed
#             sentiments.append(result["label"].upper())
#             scores.append(round(result["score"], 3))  # Confidence score
#         except Exception:
#             sentiments.append("ERROR")
#             scores.append(0.0)
#     df["finbert_sentiment"] = sentiments
#     df["finbert_confidence"] = scores
#     return df

both Finbert and Roberta

In [16]:
# Load FinBERT
finbert_model_name = "yiyanghkust/finbert-tone"
finbert_tokenizer = BertTokenizer.from_pretrained(finbert_model_name)
finbert_model = BertForSequenceClassification.from_pretrained(finbert_model_name)
finbert_pipeline = pipeline("sentiment-analysis", model=finbert_model, tokenizer=finbert_tokenizer)

# Load RoBERTa
roberta_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name)
roberta_pipeline = pipeline("sentiment-analysis", model=roberta_model, tokenizer=roberta_tokenizer)

# Combined function
def add_dual_sentiment_models(df, text_column="text"):
    finbert_sentiments = []
    finbert_scores = []
    roberta_sentiments = []
    roberta_scores = []

    for text in df[text_column].fillna("").tolist():
        try:
            fin_result = finbert_pipeline(text[:512])[0]
            finbert_sentiments.append(fin_result["label"].upper())
            finbert_scores.append(round(fin_result["score"], 3))
        except Exception:
            finbert_sentiments.append("ERROR")
            finbert_scores.append(0.0)

        # try:
        #     rob_result = roberta_pipeline(text[:512])[0]
        #     roberta_sentiments.append(rob_result["label"].upper())
        #     roberta_scores.append(round(rob_result["score"], 3))
        # except Exception:
        #     roberta_sentiments.append("ERROR")
        #     roberta_scores.append(0.0)

    df["finbert_sentiment"] = finbert_sentiments
    df["finbert_confidence"] = finbert_scores
    # df["roberta_sentiment"] = roberta_sentiments
    # df["roberta_confidence"] = roberta_scores

    return df

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Device set to use cpu


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Device set to use cpu


In [17]:


# Use lightweight spaCy
nlp = spacy.blank("en")

KEY_METRICS = {
    "EPS": ["eps", "earnings per share"],
    "NII": ["nii", "net interest income"],
    "ROE": ["roe", "return on equity"],
    "ROA": ["roa", "return on assets"],
    "NIM": ["nim", "net interest margin"],
    "LIQUIDITY": ["liquidity", "lcr", "liquidity coverage ratio", "wholesale funding"],
    "CAPITAL": ["capital", "cet1", "tcr", "total capital ratio"],
    "PROFITABILITY": ["profitability"],
    "PROVISIONS": ["provisions", "pll", "loan loss", "provision for losses"],
    "NET INCOME": ["net income"],
    "ASSET QUALITY": ["asset quality", "npl", "non performing loan"]
}

# Words that imply performance movement or quantification
QUALIFIERS = [
    "increase", "increased", "decrease", "decreased", "decline", "declined",
    "improve", "improved", "growth", "grew", "dropped", "fell", "stable",
    "stabilized", "unchanged", "flat", "rose", "risen", "higher", "lower",
    "million", "billion", "%"
]


# === Combined Metric + % Value Extraction Function ===
def extract_structured_financials(pdf_path):
    results = []

    fname = os.path.basename(pdf_path)
    match = re.search(r"Q(\d)[_\s-]?(\d{4})", fname)
    quarter, year = (int(match.group(1)), int(match.group(2))) if match else (None, None)

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue

            lines = text.split('\n')
            for i, line in enumerate(lines):
                line_clean = line.strip()
                line_lower = f" {line_clean.lower()} "
                words_set = set(re.sub(r"[^\w\s]", "", line_lower).split())

                # === 1. Extract % values with sentence context ===
                percent_matches = re.findall(r"\d+[.,]?\d*\s?%", line)
                if percent_matches:
                    before = lines[i - 1] if i > 0 else ""
                    after = lines[i + 1] if i + 1 < len(lines) else ""
                    context = f"{before.strip()} {line_clean} {after.strip()}"
                    for match in percent_matches:
                        results.append({
                            "quarter": quarter,
                            "year": year,
                            "metric": "PERCENT VALUE",
                            "value": match.strip(),
                            "text": context.strip(),
                            "source_file": fname
                        })

                # === 2. Extract metric-related values with number or qualifier ===
                for metric_group, aliases in KEY_METRICS.items():
                    if any(f" {alias} " in line_lower for alias in aliases):  # enforce word boundary
                        has_number = bool(re.search(r"\d+[.,]?\d*", line))
                        has_qualifier = any(q in line_lower for q in QUALIFIERS)

                        if not has_number and not has_qualifier:
                            continue  # skip vague mentions

                        match = re.search(r"(\d+[.,]?\d*)\s?(million|billion|%)?", line, re.IGNORECASE)
                        value = match.group(0).strip() if match else None

                        results.append({
                            "quarter": quarter,
                            "year": year,
                            "metric": metric_group,
                            "value": value,
                            "text": line_clean,
                            "source_file": fname
                        })

    df = pd.DataFrame(results)
    if not df.empty:
        # df = add_finbert_sentiment(df)
        df = add_dual_sentiment_models(df)

    return df


In [18]:

nlp = spacy.load("en_core_web_sm")

# Add known banks — extend as needed
org_patterns = [
    {"label": "ORG", "pattern": "Bank of America"},
    {"label": "ORG", "pattern": "Goldman Sachs"},
    {"label": "ORG", "pattern": "JPMorgan"},
    {"label": "ORG", "pattern": "Morgan Stanley"},
    {"label": "ORG", "pattern": "UniCredit"},
    {"label": "ORG", "pattern": "Deutsche Bank"},
    {"label": "ORG", "pattern": "HSBC"},
    {"label": "ORG", "pattern": "UBS"},
    {"label": "ORG", "pattern": "Citi"},
    {"label": "ORG", "pattern": "BNP Paribas"},
    {"label": "ORG", "pattern": "ING"},
    {"label": "ORG", "pattern": "Credit Suisse"},
    {"label": "ORG", "pattern": "Barclays"},
    {"label": "ORG", "pattern": "Mediobanca"},
    {"label": "ORG", "pattern": "Credit Agricole"},
    {"label": "ORG", "pattern": "Autonomous Research"},
    {"label": "ORG", "pattern": "BNP"},
    {"label": "ORG", "pattern": "Citi"},
    {"label": "ORG", "pattern": "UBS"},
    {"label": "ORG", "pattern": "Jefferies"},
    {"label": "ORG", "pattern": "KBW"},
    {"label": "ORG", "pattern": "Bank of America"},
    {"label": "ORG", "pattern": "Morgan Stanley"},
]



def extract_person_and_org_from_operator(text):
    doc = nlp(text)
    person = None
    org = None

    # Try spaCy NER first
    for ent in doc.ents:
        if ent.label_ == "PERSON" and not person:
            person = ent.text
        elif ent.label_ == "ORG" and not org:
            org = ent.text

    # Fallback regex: e.g. "from John Smith of Goldman Sachs"
    if not person or not org:
        match = re.search(r"from\s+(.*?)\s+of\s+([A-Za-z\s&]+)", text, re.IGNORECASE)
        if match:
            if not person:
                person = match.group(1).strip()
            if not org:
                org = match.group(2).strip()

    return person, org

In [19]:
def extract_qna_long_format(pdf_path):
    MANAGEMENT_NAMES = ["Andrea Orcel", "Stefano Porro"]

    def is_speaker_line(line):
        return bool(re.match(r"^[A-Z][a-z]+(\s[A-Z][a-z]+)?$", line.strip())) or line.strip() in MANAGEMENT_NAMES or line.strip() == "Operator"

    fname = os.path.basename(pdf_path)
    match = re.search(r"Q(\d)[_\s-]?(\d{4})", fname)
    quarter, year = (int(match.group(1)), int(match.group(2))) if match else (None, None)

    with pdfplumber.open(pdf_path) as pdf:
        full_text = "\n".join(p.extract_text() for p in pdf.pages if p.extract_text())

    if "Question-and-Answer Session" not in full_text:
        return pd.DataFrame()

    qa_text = full_text.split("Question-and-Answer Session", 1)[1]
    lines = qa_text.split("\n")

    blocks = []
    current_speaker = None
    current_block = []

    for line in lines:
        line = line.strip()
        if is_speaker_line(line):
            if current_speaker and current_block:
                blocks.append({"speaker": current_speaker, "text": " ".join(current_block).strip()})
            current_speaker = line
            current_block = []
        else:
            current_block.append(line)

    if current_speaker and current_block:
        blocks.append({"speaker": current_speaker, "text": " ".join(current_block).strip()})

    entries = []
    speaker_to_org = {}
    current_question = None
    question_number = 0

    for i in range(len(blocks)):
        block = blocks[i]
        speaker = block["speaker"]

        if speaker == "Operator":
            # Try to extract person/org from the operator’s introduction
            person, org = extract_person_and_org_from_operator(block["text"])
            if person and org:
                speaker_to_org[person] = org

            # Look ahead to find the actual question
            if i + 1 < len(blocks):
                next_block = blocks[i + 1]
                if next_block["speaker"] not in MANAGEMENT_NAMES and next_block["speaker"] != "Operator":
                    question_number += 1
                    current_question = next_block["speaker"]
                    entries.append({
                        "quarter": quarter,
                        "year": year,
                        "question_number": question_number,
                        "speaker": current_question,
                        "institution": speaker_to_org.get(current_question),
                        "type": "Question",
                        "text": next_block["text"],
                        "source_file": fname
                    })

        elif speaker in MANAGEMENT_NAMES and current_question is not None:
            entries.append({
                "quarter": quarter,
                "year": year,
                "question_number": question_number,
                "speaker": speaker,
                "institution": "UniCredit",  # known from context
                "type": "Answer",
                "text": block["text"],
                "source_file": fname
            })

    return pd.DataFrame(entries)

Loop

In [20]:
all_financial_dfs = []
all_qna_long_dfs = []

pdf_files = [f for f in os.listdir(transcript_folder) if f.lower().endswith(".pdf")]

for filename in tqdm(pdf_files, desc="Processing transcripts", unit="file"):
    full_path = os.path.join(transcript_folder, filename)

    try:
        # === Financial Extraction ===
        df_fin = extract_structured_financials(full_path)
        if not df_fin.empty:
            all_financial_dfs.append(df_fin)

        # === Q&A Extraction + Sentiment ===
        df_qna = extract_qna_long_format(full_path)
        if not df_qna.empty:
            # df_qna = add_finbert_sentiment(df_qna)
            df_qna = add_dual_sentiment_models(df_qna)

            all_qna_long_dfs.append(df_qna)

    except Exception as e:
        print(f"❌ Error processing {filename}: {e}")

# === MERGE AND SAVE FINANCIAL DATA ===
if all_financial_dfs:
    df_fin_merged = pd.concat(all_financial_dfs, ignore_index=True)
    fin_excel = os.path.join(output_folder, "merged_financial_sentences_finbertonly.xlsx")
    fin_pickle = os.path.join(output_folder, "merged_financial_sentences_finbertonly.pkl")
    df_fin_merged.to_excel(fin_excel, index=False)
    df_fin_merged.to_pickle(fin_pickle)
    print(f"✅ Financial data saved to:\n- {fin_excel}\n- {fin_pickle}")
else:
    print("⚠️ No financial sentences were extracted.")

# === MERGE AND SAVE Q&A DATA ===
if all_qna_long_dfs:
    df_qna_merged = pd.concat(all_qna_long_dfs, ignore_index=True)
    qna_excel = os.path.join(output_folder, "merged_qna_long_finbertonly.xlsx")
    qna_pickle = os.path.join(output_folder, "merged_qna_long_finbertonly.pkl")
    df_qna_merged.to_excel(qna_excel, index=False)
    df_qna_merged.to_pickle(qna_pickle)
    print(f"Q&A data saved to:\n- {qna_excel}\n- {qna_pickle}")
else:
    print("No Q&A data extracted.")


Processing transcripts: 100%|██████████| 9/9 [12:49<00:00, 85.46s/file]


✅ Financial data saved to:
- /content/drive/My Drive/BoE/Data/Unicredit/extracted_outputs/merged_financial_sentences_finbertonly.xlsx
- /content/drive/My Drive/BoE/Data/Unicredit/extracted_outputs/merged_financial_sentences_finbertonly.pkl
Q&A data saved to:
- /content/drive/My Drive/BoE/Data/Unicredit/extracted_outputs/merged_qna_long_finbertonly.xlsx
- /content/drive/My Drive/BoE/Data/Unicredit/extracted_outputs/merged_qna_long_finbertonly.pkl


update ROBERTA labels

In [21]:
# qna_excel = os.path.join(output_folder, "merged_qna_long.xlsx")
# fin_excel = os.path.join(output_folder, "merged_financial_sentences.xlsx")

# # Label map for RoBERTa
# label_map = {
#     "LABEL_0": "NEGATIVE",
#     "LABEL_1": "NEUTRAL",
#     "LABEL_2": "POSITIVE"
# }

# # === Q&A file update ===
# df_qna = pd.read_excel(qna_excel)
# if "roberta_sentiment" in df_qna.columns:
#     df_qna["roberta_sentiment"] = df_qna["roberta_sentiment"].map(label_map).fillna(df_qna["roberta_sentiment"])
#     df_qna.to_excel(qna_excel, index=False)
#     print("✅ Q&A sentiment labels updated.")

# # === Financial file update ===
# df_fin = pd.read_excel(fin_excel)
# if "roberta_sentiment" in df_fin.columns:
#     df_fin["roberta_sentiment"] = df_fin["roberta_sentiment"].map(label_map).fillna(df_fin["roberta_sentiment"])
#     df_fin.to_excel(fin_excel, index=False)
#     print("✅ Financial sentiment labels updated.")

✅ Q&A sentiment labels updated.
✅ Financial sentiment labels updated.


# Topic extraction installation

In [24]:
drive.mount('/content/drive')
!ls '/content/drive/My Drive/BoE'

output_folder = '/content/drive/My Drive/BoE/Data/Unicredit/extracted_outputs/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Codes				'Presentation Documents'
 Data				 Project_Plan
'Ideas and Research Documents'	'Russell Hunter Notebooks'
 Meetings			 Video_presentation
'Other Documents'


In [25]:
df_fin_merged = pd.read_excel(output_folder + "merged_financial_sentences_finbertonly.xlsx")
df_qna_merged = pd.read_excel(output_folder + "merged_qna_long_finbertonly.xlsx")

import more packages

In [26]:
!pip install keybert sentence-transformers nltk openpyxl

import pandas as pd
import re
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm

## Topic extraction code

In [32]:
# Download required resources
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

# Load custom embedding model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
kw_model = KeyBERT(model=embedding_model)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [33]:
# Function words
function_stopwords = [
    "we", "have", "has", "had", "were", "was", "are", "is", "it's", "its", "being", "been",
    "do", "does", "doing", "did", "can", "may", "might", "come", "comes", "going"
]

qa_filler_phrases = [
    "thanks", "thank you", "appreciate", "question", "questions", "ask", "asking", "follow-up",
    "couple of questions", "good morning", "good afternoon", "hello", "thanks for taking my question",
    "thanks for the call", "thank you for your time", "joining us", "line is open", "line",
    "hand over", "presentation", "prepared remarks", "pick up with team", "management remarks",
    "closing remarks", "just wondering", "can you talk a bit more about", "could you elaborate",
    "as you mentioned", "as you said", "i guess", "i wonder if", "may i ask", "i was going to ask",
    "one more question", "a quick clarification", "sorry to go on", "very helpful", "hope that makes sense", "indiscernible",
    "Andrea", "Stefano", "yes", "discussed", "tkank you", "very much"
]



# Combine all stopwords
lemmatizer = WordNetLemmatizer()
base_stopwords = set(stopwords.words("english"))
domain_stopwords = set(qa_filler_phrases  + function_stopwords)
all_stopwords = base_stopwords.union(domain_stopwords)

In [34]:
def clean_text(text):
    # Lowercase and remove special characters
    text = re.sub(r"[^\w\s]", " ", text.lower())
    words = text.split()
    # Lemmatize and remove stopwords
    clean_words = [
        lemmatizer.lemmatize(w)
        for w in words
        if w not in all_stopwords and len(w) > 2
    ]
    return " ".join(clean_words)

In [38]:
def add_topic_keywords(df, text_column="text", top_n=3):
    cleaned_texts = df[text_column].fillna("").astype(str).apply(clean_text)
    topics = []

    for text in tqdm(cleaned_texts, desc=f"Extracting topics for '{text_column}'", unit="row"):
        try:
            keywords = kw_model.extract_keywords(
                text,
                keyphrase_ngram_range=(1, 2),
                stop_words=None,
                top_n=top_n
            )
            topic_list = [kw[0] for kw in keywords]
            topics.append(", ".join(topic_list))
        except Exception:
            topics.append("ERROR")

    df["topics"] = topics
    return df

In [39]:
df_fin_topic= add_topic_keywords(df_fin_merged, text_column="text")
df_qna_topic = add_topic_keywords(df_qna_merged, text_column="text")


df_qna_topic.to_excel(output_folder + 'qna_topics.xlsx', index=False)
df_qna_topic.to_pickle(output_folder +  'qna_topics.pkl')

df_fin_topic.to_excel(output_folder + 'financial_topics.xlsx', index=False)
df_fin_topic.to_pickle(output_folder + 'financial_topics.pkl')

Extracting topics for 'text': 100%|██████████| 205/205 [03:02<00:00,  1.12row/s]
